In [40]:
%load_ext autoreload
%autoreload 2

from sentence_transformers import SentenceTransformer
from src.text_embedder.embedder import text_to_embedding_transformer

import pandas as pd
from loguru import logger

from src.text_embedder.embedder import EmbeddingsModel

from pathlib import Path
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Data

In [41]:
notebook_path = Path().resolve().parent
print(notebook_path)

# Construct the filename relative to the new path
filename = notebook_path / "test_frames_ch7.csv"

# Read the CSV file
df = pd.read_csv(filename)
print(df)

C:\Users\baatout\PycharmProjects\afm-vlm\experiments\second_experiment
    ID                                             Prompt Length  GT_Keyframe
0    1                                 What are feedlots?  short           47
1    2                                What is weathering?  short           17
2    3  What is desertification and how much productiv...   long           22
3    4  What is the conservation reserve programm of 1...   long           29
4    5          GM crops today are engineered to resist:    long           56
5    6  90% of energy is lost from one trophic level t...   long           45
6    7                               Risks of biocontrol?  short           40
7    8  As the populations increases pesticides lose t...   long           38
8    9  Substainable fertilizer use involves monitorin...   long           36
9   10                               What is overgrazing?  short           29
10  11                   Why does the USDA issues labels?  short       

In [42]:
# create dataframe to save results of the experiments
columns = ['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3']

df_test = pd.DataFrame(columns=columns)

df_ocr_only = df_test
df_ocr_lava = df_test
df_ocr_transcriptions = df_test

df_short_llm_summary = df_test
df_extensive_summary = df_test

In [43]:
def add_to_df(prompt, gt_keyframe, result):
    # Create a new row with the provided data
    return {
        'Prompt': prompt,
        'GT_Keyframe': gt_keyframe,
        'Top_1': extract_keyframe_number(result[0]) if len(result) > 0 else None,
        'Top_2': extract_keyframe_number(result[1]) if len(result) > 1 else None,
        'Top_3': extract_keyframe_number(result[2]) if len(result) > 2 else None
    }


# ietrate over the dataframe and get the results
def get_results(df):
    for _, row in df.iterrows():
        logger.info(row['Prompt'])
        prompt = row['Prompt']
        gt_keyframe = row['GT_Keyframe']

        # Search for similar images
        output = embedding_model.search_similar_images_top_k(prompt, gt_keyframe, 3)
        res_row = add_to_df(prompt, gt_keyframe, output)
        rows.append(res_row)

    return rows


def extract_keyframe_number(path):
    """
    Extracts the scene number from the given file path.

    Parameters:
    path (str): The full path of the file.

    Returns:
    str: The extracted scene number.
    """
    # Get filename without extension
    filename = os.path.splitext(os.path.basename(path))[0]

    # Extract '032' from filename
    scene_number = filename.split('-Scene-')[-1].split('-')[0]

    return scene_number

In [44]:
#top_3 = []
#embedding_model.check_proximity_keyframes(90, top_3)

# LOAD MODEL 

In [45]:
embedding_model = EmbeddingsModel()

<Figure size 800x2000 with 0 Axes>

In [46]:
csv = notebook_path / "ch7_soil_agriculture.csv"
df_data_extensive = pd.read_csv(csv)
df_data_extensive

,Unnamed: 0,47,59,24,55,28,36,63,71,12,...,68,15,33,50,21,42,17,9,5,66
0,img_path,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...
1,timestamps,"[2538.0, 2567.0]","[2965.4, 2975.2]","[1457.9, 1482.9]","[2870.2, 2884.6]","[1580.1, 1702.6]","[2012.5, 2082.4]","[3118.6, 3164.3]","[3347.2, 3387.0]","[738.9, 813.4]",...,"[3279.2, 3294.1]","[882.5, 957.0]","[1873.0, 1946.0]","[2660.3, 2689.7]","[1288.5, 1374.7]","[2370.3, 2436.3]","[1045.1, 1116.5]","[444.7, 493.4]","[261.1, 329.7]","[3229.6, 3253.0]"
2,transcription,Feed lots are factory farms are also called c...,Resistance to the weed killer glyphosate is b...,The Dust Bowl prompted the U.S. to fight eros...,Biotechnology is transforming the products ar...,By increasing organic matter and soil biodive...,Sustainable approaches to fertilizing deliver...,The growth of sustainable agriculture. Susta...,"In weighing the issues, here are some items t...",The desire for greater quantity and quality o...,...,Modern's team found that soil and organic plo...,We can't simply keep expanding agriculture in...,Sustainable approaches to irrigation maximize...,The benefits are a reduced pressure on overha...,maintaining healthy soils. Soil degradation ...,Polynators are beneficial bugs worth preservi...,Soil formation begins when the lithosphere's ...,"If on Earth, we're going to enhance global fo...","Our goal will be food security, meaning that ...",The US 2014 Farm Bill has provisions to direc...
3,ocr_extracted_text,+ Feedlots (factory farms)\n* Also called conc...,Resistance to the weed-\n\nkiller glyphosate i...,* Soil Conservation Service (SCS)\n* Started i...,Biotec’\n\naround\n\nnology is transform\n\nth...,Sustainable agriculture begins with soil\nmana...,* Sustainable approaches to fertilizing delive...,Sustainable agriculture keeps high crop yields...,* Do You Want Food Labeled?\n\n* The USDA issu...,» Green Revolution\n* Drastically increased fo...,...,The Science Behind the Story (cont’d)\n\n» How...,» Suitable farmland is disappearing\n+ We must...,* Match crops and climate\n\n* Don't plant cro...,* Benefits\n* Reduces pressure on overharveste...,* People make land more vulnerable to erosion ...,Not all insects are pests; some are absolutely...,» Parent material\n+ The base geologic materia...,» Agriculture\n\n+ Practice of raising crops a...,"* Food security\n* Guarantee of an adequate, s...",* The U.S. 2014 Farm Bill has provisions to di...
4,llava_result,The image you've provided appears to be a sli...,The image you've provided appears to be a sli...,The image you've provided appears to be a scr...,The image you've provided appears to be a sli...,The slide appears to be from an academic lect...,The image you've provided appears to be a sli...,The image you've provided appears to be a sli...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,...,The image you've provided appears to be a sli...,The image you've provided appears to be a sli...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The im

In [47]:
# get img_path
img_path = df_data_extensive.iloc[0].to_dict().values()

# convert to list
img_path = list(img_path)
img_path = img_path[1:]
logger.info(img_path)

2024-07-11 11:30:40.705 | INFO     | __main__:<module>:7 - ['/Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/ch7_soil_agriculture/extracted_keyframes/ch7_soil_agriculture-Scene-047-01.jpg', '/Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/ch7_soil_agriculture/extracted_keyframes/ch7_soil_agriculture-Scene-059-01.jpg', '/Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/ch7_soil_agriculture/extracted_keyframes/ch7_soil_agriculture-Scene-024-01.jpg', '/Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/ch7_soil_agriculture/extracted_keyframes/ch7_soil_agriculture-Scene-055-01.jpg', '/Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/ch7_soil_agriculture/extracted_keyframes/ch7_soil_agriculture-Scene-028-01.jpg', '/Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/ch7_soil_agriculture/extracted_keyframes/ch7_soil_agriculture-Scene-036-01.jpg', '/Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/ch7_soil_agriculture/extracted_keyframes/ch7_soil_agriculture-Scene-063-01.jpg', '/Users/m

In [48]:
# get llm_long_summary 
llm_long_summary = df_data_extensive.iloc[6].to_dict().values()

# convert to list
llm_long_summary = list(llm_long_summary)
llm_long_summary = llm_long_summary[1:]
logger.info(llm_long_summary)

2024-07-11 11:30:41.022 | INFO     | __main__:<module>:7 - ["Feedlots, also called concentrated animal feeding operations (CAFOs), are huge warehouses or pens that deliver food to animals living at extremely high densities. They produce over half of the world's pork and most of its poultry. The benefits include allowing greater production of affordable meat and reducing grazing impacts on land.", 'Resistance to glyphosate, a toxic chemical used as a herbicide, is being documented more widely than in the past. Glyphosate-resistant species are emerging, with over 900 studies documenting its harmful effects in the past 20 years. The use of glyphosate has increased significantly over time, particularly after the introduction of genetically modified foods. This raises concerns about the potential health risks associated with consuming GM foods and highlights the need for regulation and monitoring of glyphosate usage.', 'The Soil Conservation Service (SCS) was started in 1935 by Congress to 

In [49]:
# get ocr_only
# Since ocr_only embeddings can't be mapped to floats due to e-02, e-01, we're generating new embeddings 
ocr_only = df_data_extensive.iloc[3].to_dict().values()

# convert to list
ocr_only = list(ocr_only)
ocr_only = ocr_only[1:]
logger.info(ocr_only)

2024-07-11 11:30:41.327 | INFO     | __main__:<module>:8 - ['+ Feedlots (factory farms)\n* Also called concentrated animal feeding operations\n(CAFOs)\n+ Huge warehouses or pens deliver food to animals living\nat extremely high densities\n* Produce over half of the world’s pork and most of its\npoultry\n* Benefits\n* Allow greater production of affordable meat\n* Reduce grazing impacts on the land\n\n', 'Resistance to the weed-\n\nkiller glyphosate is being\ndocumented more widely\nthan in the past\n\nClyphosateesistant species\n\n', '* Soil Conservation Service (SCS)\n* Started in 1935 by Congress\n* Works with farmers to develop conservation plans for\nfarms\n+ Now named the Natural Resources Conservation\nService\nOther countries started their own conservation\nagencies\n\n', 'Biotec’\n\naround\n\nnology is transform\n\nthe produ\n\n* GM foods have gone from science fiction to\nmainstream agriculture in the last 3 decades\n\n180 5\n\n460 | — World total ;\n440 | — Industrialized nat

In [50]:
# get ocr_lava 
lava_only = df_data_extensive.iloc[4].to_dict().values()

lava_only = list(lava_only)

lava_only = lava_only[1:]

concat_result = []

# list to a list of strings
ocr_only = [str(i) for i in ocr_only]

lava_only = [str(i) for i in lava_only]

for i in range(len(ocr_only)):
    concat_result.append(str(ocr_only[i]) + str(lava_only[i]))


In [51]:
# get ocr_transcriptions
transcriptions_only = df_data_extensive.iloc[2].to_dict().values()
ocr_only = df_data_extensive.iloc[3].to_dict().values()

ocr_only = list(ocr_only)
ocr_only = ocr_only[1:]

transcriptions_only = list(transcriptions_only)
transcriptions_only = transcriptions_only[1:]

ocr_transcriptions = []
# convert to list
for i in range(len(ocr_only)):
    ocr_transcriptions.append(str(ocr_only[i]) + str(transcriptions_only[i]))

# Embedd with LLM long summary

In [52]:
# Embedd with LLM long summary
logger.info(f"Embedd with LLM long summary")

result = []

embedding_model = EmbeddingsModel()

embedding_model.text_embeddings = None

embedding_model.img_paths = None

embedding_model.img_paths = img_path

embeddings = embedding_model.generate_dataset_embeddings_standard_tokenizer(llm_long_summary)

embedding_model.text_embeddings = embeddings

logger.info(f"Embedding Model.text_embeddings: {embedding_model.text_embeddings}")

rows = []

rows = get_results(df)

df_extensive_summary = pd.DataFrame(rows, columns=['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3'])

# Save dataframe to CSV
df_extensive_summary.to_csv('df_standard_llm_long_summary_ch7.csv', index=False)


2024-07-11 11:30:42.507 | INFO     | __main__:<module>:2 - Embedd with LLM long summary
2024-07-11 11:30:52.706 | INFO     | __main__:<module>:18 - Embedding Model.text_embeddings: tensor([[-0.3105, -0.2126, -0.0940,  ...,  0.2942,  0.2064, -0.2412],
        [-0.2912,  0.0772, -0.0872,  ..., -0.1356,  0.1837,  0.0590],
        [-0.4012, -0.1009, -0.4453,  ...,  0.1038,  0.0645,  0.0591],
        ...,
        [ 0.2111,  0.2091,  0.0063,  ..., -0.0335, -0.0541,  0.1448],
        [ 0.0241,  0.1050, -0.2728,  ..., -0.1618, -0.1975, -0.0643],
        [ 0.1189,  0.1825, -0.1903,  ..., -0.1839,  0.3104, -0.0752]])
2024-07-11 11:30:52.710 | INFO     | __main__:get_results:15 - What are feedlots?
2024-07-11 11:30:52.781 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:175 - Top 3 Similarity scores: torch.return_types.topk(
values=tensor([0.6497, 0.4601, 0.2465]),
indices=tensor([ 0, 52, 45])) - GT is keyframe number 47
2024-07-11 11:30:52.783 | INFO     | src.text_embedder.em

<Figure size 800x2000 with 0 Axes>

# Embedd with OCR Only 

In [55]:
# Embedd with OCR ONLY 
logger.info(f"Embedd with OCR ONLY ")

result = []

embedding_model = EmbeddingsModel()

embedding_model.text_embeddings = None

embedding_model.img_paths = None

embedding_model.img_paths = img_path


# need to be embedded

embeddings = embedding_model.generate_dataset_embeddings_standard_tokenizer(ocr_only)

embedding_model.text_embeddings = embeddings

rows = []

rows = get_results(df)

df_ocr_only = pd.DataFrame(rows, columns=['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3'])

# Save dataframe to CSV
df_ocr_only.to_csv('df_standard_ocr_only_ch7.csv', index=False)
logger.info('Saved df_standard_ocr_only_ch7.csv')

2024-07-11 11:31:23.351 | INFO     | __main__:<module>:2 - Embedd with OCR ONLY 
2024-07-11 11:31:29.965 | INFO     | __main__:get_results:15 - What are feedlots?
2024-07-11 11:31:30.040 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:175 - Top 3 Similarity scores: torch.return_types.topk(
values=tensor([0.4588, 0.3192, 0.2023]),
indices=tensor([ 0, 52, 20])) - GT is keyframe number 47
2024-07-11 11:31:30.042 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:177 - Length of img paths: 73
2024-07-11 11:31:30.043 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:185 - #####GT is keyframe number 47#####
2024-07-11 11:31:30.045 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:186 - Max similarity for index 0 is the keyframe /Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/ch7_soil_agriculture/extracted_keyframes/ch7_soil_agriculture-Scene-047-01.jpg
2024-07-11 11:31:30.047 | INFO     | src.text_embedder.embedder:s

<Figure size 800x2000 with 0 Axes>

# Embedd with OCR TRANSCRIPTIONS

In [56]:
# Embedd with OCR TRANSCRIPTIONS
# TODO: Doesn't work since transcriptions are nans
logger.info(f"Embedd with TRANSCRIPTIONS")

result = []

embedding_model = EmbeddingsModel()

embedding_model.text_embeddings = None

embedding_model.img_paths = None

embedding_model.img_paths = img_path

embeddings = embedding_model.generate_dataset_embeddings_standard_tokenizer(ocr_transcriptions)

embedding_model.text_embeddings = embeddings

logger.info(f"Embedding Model.text_embeddings: {embedding_model.text_embeddings}")

rows = []

rows = get_results(df)

df_ocr_transcriptions = pd.DataFrame(rows, columns=['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3'])

# Save dataframe to CSV
df_ocr_transcriptions.to_csv('df_standard_df_ocr_transcriptions_ch7.csv', index=False)

2024-07-11 11:31:32.280 | INFO     | __main__:<module>:3 - Embedd with TRANSCRIPTIONS
2024-07-11 11:31:43.954 | INFO     | __main__:<module>:19 - Embedding Model.text_embeddings: tensor([[-4.0239e-02, -9.7657e-02,  7.2754e-02,  ...,  8.3158e-02,
          4.2901e-01, -1.0000e-01],
        [-1.7349e-01,  1.1662e-01, -1.8165e-01,  ..., -3.0387e-02,
          2.6441e-01,  2.4149e-01],
        [-2.7467e-01, -1.0692e-01, -2.1019e-01,  ...,  1.3203e-01,
          1.1352e-01,  4.4427e-02],
        ...,
        [ 1.3807e-01,  1.0964e-01,  1.3175e-02,  ..., -7.8767e-02,
          7.5188e-02,  7.9969e-02],
        [ 3.0522e-02,  1.4148e-01, -6.2367e-02,  ..., -2.1110e-01,
         -8.1740e-02,  1.0928e-04],
        [ 1.0573e-01,  1.7724e-01, -1.2145e-01,  ..., -1.9275e-01,
          3.3453e-01, -6.5827e-02]])
2024-07-11 11:31:43.957 | INFO     | __main__:get_results:15 - What are feedlots?
2024-07-11 11:31:44.015 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:175 - Top 3 Sim

<Figure size 800x2000 with 0 Axes>

# Embedd with OCR Lava

In [57]:
# Embedd with OCR LAVA
logger.info(f"Embedd with OCR LAVA")

result = []

embedding_model = EmbeddingsModel()

embedding_model.text_embeddings = None

embedding_model.img_paths = None

embedding_model.img_paths = img_path

# get the embedder model
embedder_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

concat_result_embeddings = text_to_embedding_transformer(concat_result, embedder_model)

embedding_model.text_embeddings = concat_result_embeddings

logger.info(embedding_model.text_embeddings)

rows = []

rows = get_results(df)

df_ocr_lava = pd.DataFrame(rows, columns=['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3'])

# Save dataframe to CSV
df_ocr_lava.to_csv('df_standard_ocr_lava_ch7.csv', index=False)

2024-07-11 11:31:46.221 | INFO     | __main__:<module>:2 - Embedd with OCR LAVA
2024-07-11 11:32:01.369 | INFO     | __main__:<module>:21 - tensor([[-0.1126,  0.0052,  0.0602,  ...,  0.0665,  0.2792, -0.0277],
        [-0.2267,  0.0760, -0.0226,  ..., -0.0584, -0.0022,  0.1039],
        [-0.3232, -0.0950, -0.1765,  ...,  0.1683,  0.0337,  0.1378],
        ...,
        [ 0.1165, -0.0265,  0.1186,  ...,  0.1997,  0.0019,  0.2035],
        [ 0.0530,  0.0933, -0.0790,  ..., -0.1557, -0.0689,  0.0544],
        [ 0.0831,  0.0933, -0.1045,  ..., -0.0659,  0.2768, -0.0006]])
2024-07-11 11:32:01.374 | INFO     | __main__:get_results:15 - What are feedlots?
2024-07-11 11:32:01.438 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:175 - Top 3 Similarity scores: torch.return_types.topk(
values=tensor([0.4421, 0.3625, 0.3608]),
indices=tensor([ 0, 45, 52])) - GT is keyframe number 47
2024-07-11 11:32:01.440 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:177 - 

<Figure size 800x2000 with 0 Axes>